In [8]:
import pandas as pd
import glob as gb
import os

files = gb.glob('./data/Ward_Age/*')
dataname = '町丁目名年齢年代別人口分布'

d=os.getcwd()
prefix = d + '/../data/Ward_Age/'

bin_band =  list(range(0,131,5))

g_code = pd.read_csv(prefix+'東京都人口世帯数_estat.txt', encoding='SHIFT-JIS', dtype={'KEY_CODE':str})

g_code[g_code['HYOSYO']>3]
g_code=g_code[['KEY_CODE','NAME','CITYNAME']]

## 大田区

In [9]:
ohta_file = prefix + '大田区2022年1月_rev.xlsx'

ohta_data = pd.read_excel(ohta_file).set_index('年齢')

ohta_finedata = pd.DataFrame(ohta_data.stack()).reset_index().set_axis(['年齢','町丁目名','人口'],axis=1)

ohta_finedata = ohta_finedata[['町丁目名','年齢','人口']]

ohta_finedata['町丁目名'] = ohta_finedata['町丁目名'].str.replace('　','')

ohta_finedata.to_excel(prefix + dataname + '（大田）.xlsx', index=False)

In [10]:
ew=pd.cut(ohta_finedata['年齢'], bin_band, right=False)
#pd.DataFrame(pd.DataFrame(ew).value_counts()).sort_values('年齢')

In [11]:
pd.merge(ohta_finedata, g_code, left_on='町丁目名', right_on='NAME', how='left')

,町丁目名,年齢,人口,KEY_CODE,NAME,CITYNAME
0,大森東一丁目,0,24,13111002001,大森東一丁目,大田区
1,大森東二丁目,0,34,13111002002,大森東二丁目,大田区
2,大森東三丁目,0,30,13111002003,大森東三丁目,大田区
3,大森東四丁目,0,35,13111002004,大森東四丁目,大田区
4,大森東五丁目,0,27,13111002005,大森東五丁目,大田区
...,...,...,...,...,...,...
29031,新蒲田二丁目,121,0,13111053002,新蒲田二丁目,大田区
29032,新蒲田三丁目,121,0,13111053003,新蒲田三丁目,大田区
29033,ふるさとの浜辺,121,0,NaN,NaN,NaN
29034,令和島一丁目,121,0,NaN,NaN,NaN


### 品川

In [12]:
male=list(range(0,241,2))
female=list(range(1,241,2))

shina_file = prefix + '品川区2023年1月_rev.xls'
shina_data = pd.read_excel(shina_file, dtype={'行政コード':int})

shina_m = shina_data.loc[:,['行政コード','丁目名']+male].set_index(['行政コード','丁目名']).stack()
shina_f = shina_data.loc[:,['行政コード','丁目名']+female].set_index(['行政コード','丁目名']).stack()

shina_m_fine= pd.DataFrame(shina_m).reset_index().rename(columns={'level_2':'age',0:'人口'})
shina_m_fine['年齢']=shina_m_fine['age']//2

shina_f_fine= pd.DataFrame(shina_f).reset_index().rename(columns={'level_2':'age',0:'人口'})
shina_f_fine['年齢']=(shina_f_fine['age'])//2

scols=['行政コード','丁目名','年齢','人口']
mer=['行政コード','丁目名','年齢']

shina_m_fine=shina_m_fine.drop(columns=['age'])[scols]
shina_f_fine=shina_f_fine.drop(columns=['age'])[scols]


shina_finedata = pd.merge(shina_m_fine, shina_f_fine, on=mer)

#for t,i in shina_finedata.iterrows():
#    shina_finedata.loc[i,'丁目名'] = shina_finedata.loc[i, '丁目名'].replace('\u3000','')
#    print(shina_finedata.loc[i, '丁目名'])

shina_finedata['人口'] = shina_finedata['人口_x']+shina_finedata['人口_y']
shina_finedata = shina_finedata.drop(columns=['人口_x','人口_y']).reset_index(drop=True)
shina_finedata = shina_finedata.rename(columns={'丁目名':'町丁目名'})
shina_finedata.to_excel(prefix + dataname + '（品川）.xlsx', index=False)

In [19]:
#from kanjize import int2kanji, kanji2int

print(shina_finedata.loc[2,'町丁目名'].replace('\u3000',''))

北品川１丁目


In [86]:
shina_finedata.loc[2,'町丁目名']

'北品川１丁目\u3000'

In [85]:
pd.merge(shina_finedata, g_code[['KEY_CODE','NAME','CITYNAME']], left_on='町丁目名', right_on='NAME', how='left')

,行政コード,町丁目名,年齢,人口,KEY_CODE,NAME,CITYNAME
0,1101,北品川１丁目,0,20,NaN,NaN,NaN
1,1101,北品川１丁目,1,17,NaN,NaN,NaN
2,1101,北品川１丁目,2,21,NaN,NaN,NaN
3,1101,北品川１丁目,3,18,NaN,NaN,NaN
4,1101,北品川１丁目,4,14,NaN,NaN,NaN
...,...,...,...,...,...,...,...
15595,6200,東八潮,115,0,NaN,NaN,NaN
15596,6200,東八潮,116,0,NaN,NaN,NaN
15597,6200,東八潮,117,0,NaN,NaN,NaN
15598,6200,東八潮,118,0,NaN,NaN,NaN


### 目黒

In [57]:
meg_file = prefix +'目黒区2022年12月_rev.xlsx'
meg_data = pd.read_excel(meg_file)
#meg_data

top = meg_data[meg_data['a']=='町丁別年齢別人口表'].reset_index()['index']
end = meg_data[meg_data['a']=='104以上'].reset_index()['index']
#print(list(top))
#print(list(end))

meg_tmp = pd.DataFrame()

#meg_data['町丁目名']=''

for k in top:
    chomei = meg_data.iloc[k+1,0].replace('　','')

    for j in list(range(k+4,k+129)):
        meg_data.loc[j,'町丁目名'] = chomei

    chodf = meg_data.iloc[(k+4):(k+129),:]
    
    chodf = chodf[~(chodf['a'].str.contains('～',na=False))]
    chodf = chodf[~(chodf['a'].str.contains('以上',na=False))]
    
    chodf = chodf.rename(columns={'a':'年齢', 'b':'人口'})
    chodf['年齢'] = chodf['年齢'].astype(int)
    
    meg_tmp = pd.concat([meg_tmp, chodf])

meg_finedata = meg_tmp[['町丁目名','年齢','人口']]

meg_finedata.to_excel(prefix + dataname + '（目黒）.xlsx', index=False)

In [58]:
meg_finedata

,町丁目名,年齢,人口
5,駒場１丁目,0,30
6,駒場１丁目,1,16
7,駒場１丁目,2,26
8,駒場１丁目,3,19
9,駒場１丁目,4,32
...,...,...,...
12129,東が丘２丁目,99,3
12131,東が丘２丁目,100,0
12132,東が丘２丁目,101,1
12133,東が丘２丁目,102,0


### 港区

In [15]:
import tabula

mina_file = prefix + '港区2022年4月.pdf'

mina_data = tabula.read_pdf(mina_file,pages="1-117",stream=True)

In [16]:
textfile = open(prefix+'港区町丁目名.txt','r')
output_text = textfile.read()
textfile.close()

import re

pattern = '(.+丁目)|(麻布.+町)|(麻布狸穴町)$'

minacho = re.findall(pattern, output_text)

from itertools import chain
def flatten(nested_list):
    return list(chain.from_iterable(nested_list))

#flatten(list(map(lambda x: list(x), minacho)))

chomeilist = list(map((lambda y: ''.join(y)), list(map(lambda x: list(x), minacho))))

In [17]:
mina_finedata = pd.DataFrame()

for i in range(0,116):
    if i != 39:
        k = mina_data[i].iloc[:21,0:3].set_axis(['世代','a','b'],axis=1)
        k['町丁目名'] = chomeilist[i]
        k = k.fillna(0)
        k = k.astype({'a':str, 'b':str})
        k['a'] = k['a'].str.replace(',','')
        k['a'] = k['a'].str.replace(r'0\.0','0', regex=True)
        k['b'] = k['b'].str.replace(',','')
        k['b'] = k['b'].str.replace(r'0\.0','0', regex=True)
    else:
        k = mina_data[i].iloc[:,0:2].set_axis(['世代','a'],axis=1)
        k['町丁目名'] = chomeilist[i]
        k['a'] = k['a'].str.replace(',','')
        k['a'] = k['a'].str.replace(r'0\.0','0', regex=True)        
        k['b'] = '0'
        
    mina_finedata = pd.concat([mina_finedata, k])

mina_finedata = mina_finedata.astype({'a':int, 'b':int}).reset_index(drop=True) 

mina_finedata['人口'] = mina_finedata['a'] + mina_finedata['b']
#

for r in mina_finedata.itertuples():
    if not(((r.a == 0) and (r.b==r.人口)) or ((r.b==0) and (r.a==r.人口))):
        print(r.町丁目名, ' ', r.a, r.b)


mina_finedata = mina_finedata.drop(columns=['a','b'])[['町丁目名','世代','人口']]
mina_finedata.to_excel(prefix + dataname + '（港）.xlsx', index=False)

### 渋谷

In [18]:
shib_data = pd.read_excel(prefix + '渋谷区2022年1月.xls', header=1)

shib_finedata = shib_data[['町丁名','年齢','人口']]

shib_finedata.to_excel(prefix + dataname + '（渋谷）.xlsx', index=False)

### 台東

In [82]:
import pandas as pd
ver = 'その２'
ocrfile = '台東区2023年1月'+ver+'.xlsx'
tai_data = pd.read_excel(prefix + ocrfile, header=53, engine='openpyxl')

/Users/mori/.pyenv/versions/3.10.6/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [83]:
tai_data #.iloc[2,3].split('\n')628

import numpy as np
#tai_tmp = pd.DataFrame([],index=list(range(0,627)), columns=list(range(0,14)))

tai_tmp = []

for r,s in tai_data.iterrows():
    if (s[0] == s[0]) and len(s[0].split('\n')) > 1:
        lt = s[0].split('\n')
        print(lt)
        l = [lt[1],''.join(lt[2:])]
    else:
        l = ['']
    
    for i in range(2,14):
        k = str(s[i]).split('\n')
        if len(k) > 2:
            l.append(k[-2])
        else:
            l.append('')

    if not len(''.join(l)) == 0:
        tai_tmp.append(l)


['行政区', '000261', '根岸1丁目']
['行政区', '000262', '根岸2丁目']
['行政区', '000263', '根岸3丁目']
['行政区', '000264', '根岸4T 目']
['一」丁', '区5--J', '政26引', '誕02岸', '行00赫1', '100根目']
['行政区', '000271', '入谷1丁目']
['行政区', '000272', '入谷2丁目']
['行政区', '000281', '竜泉1丁', '目']
['圭ロ2', '什え00IJ', 'え嶋っ】えし', '一丁心唾千｝0竜目']
['-L丁', '区3ド', '政28む', '行00参1', '手00竜目']
['行政区', '000291', '松が谷1 丁目']
['行政区', '000292', '松が谷2 丁目']
['行政区', '000293', '松が谷3', 'T目']
['行政区', '000294', '松が谷4', '丁目']
['行政区', '000302', '西浅草2', '丁目']
['行政区', '000303', '西浅草3', '丁目']
['行政区', '000311', '雷門1丁', '目']
['壬一コ1山', 'ケえっ」', '1ルりJ」ョ1', '11)nUゴLI', '21-n一且時コ『『', 'nU皇Lrト」', '■Ll』』1』曲11曲国国山山国1国山山山山山国国国山山山11ョ山1ョ1ョョョ一1国1国1国1111■11■11■IllEI」1臣」且111■且1且且且且且且1」且1王且肥且11」1」且肥11']
['行政区', '000322', '浅草2丁目']
['行政区', '000323', '浅草3丁目']
['行政区', '000324', '浅草4丁目']
['土馬J 」えり乙J耳', '正嶋3', '】丁0', '乙11ハV', '0']
['歪滴6 斤えり一ー', '記延nJャョL', '一丁10里一', '千）0曳目']
['行政区', '000327', "浅草'7丁目"]
['行政区', '000331', '花川戸1', 'T目']
['行政区', '000332', '花川戸2', '丁目']
['行政区', '000341', '千東1丁目']
['丁

In [101]:
len(gl[14])

24

In [104]:
tai_even = tai_tmp[0:101:2]
tai_odd = tai_tmp[1:101:2]

import re

gl = []

for s,t in zip(tai_even, tai_odd):
    gl.append(s + t[1:-1])


for l in range(0, len(gl)):
    for k in range(0,len(gl[0])):
        try:
            gl[l][k] = re.sub('\(.*\)|\(.*の| ', '', gl[l][k])
        except:
            print(l,'>',k)
            

14 > 24
18 > 24


25

In [105]:
tai_cols = ['行政コード','町丁目名','0~4歳', '5~9歳', '10~14歳', '15~19歳', '20~24歳', '25~29歳', '30~34歳', 
            '35~39歳', '40~44歳', '45~49歳', '50~54歳', '55~59歳', '60~64歳', '65~69歳', 
            '70~74歳', '75~79歳', '80~84歳', '85~89歳', '90~94歳', '95~99歳', '100歳~104歳',
            '100歳~104歳', '110歳以上']
len(tai_cols)


25

In [106]:
tai_finedata = pd.DataFrame(gl,columns=tai_cols)
tai_finedata.to_excel(prefix + dataname + '（台東）'+ver+'.xlsx', index=False)